# **Practical assignment for Topic 4**

**Link to your finished ipynb file in Google Colab:** *put the link here*

---

In your assigmnet, you will use the following **data**:

In [ ]:
import numpy as np
X = np.array([-3.00, -2.54, -2.08, -1.62, -1.15, -0.69, -0.23,  0.23,  0.69,  1.15,  1.62,  2.08,  2.54,  3.00]).reshape(14, 1)
y = np.array([ 2.60,  2.81,  3.49,  4.08,  3.30,  3.43,  4.59,  4.65,  4.79,  6.45,  7.58,  9.78, 12.98, 15.49]).reshape(14, 1)

Your assignment consists of the following tasks:

1. Create a total of eight polynomials from degree 1 to 8. Compute and print training MSE and validation MSE values for each of them. Validation MSE should be computed using LOOCV (use the function that we already created). Draw a plot with two curves analogous to the plot shown in slide 9 from today's lecture. Add a legend so that it is clear which curve is which.
2. Repeat Task 1 with a modified LOOCV function so that it does not use anything from scikit-learn anymore - instead it should use only numpy. You should get the same result as in Task 1.
3. Explain the plot from Task 1. What exactly do we see in it and what conclusions can we make from it?
4. Draw another plot showing all datapoints of the given dataset together with the curves of all eight polynomials from degree 1 to 8. Each curve should be in a different color. Add a legend so that it is clear which curve is which.<br>
4.a. Do the task using scikit-learn.<br>
4.b. Do the task using numpy.

*While doing the tasks you are allowed to use only those function libraries and only those functions of numpy and scikit-learn libraries that we already have used in our notebooks of the course.*

**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**


## Question 1.

LOOCV function that we created in class :

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score
loo = LeaveOneOut()
def loocv(X, y, model):
  loo = LeaveOneOut()  # define LOOCV

  yHat = np.empty(y.size)  # this will be filled with predictions for us to compute scores after the loop

  for train_idx, validation_idx in loo.split(X, y):  # loop through all folds
    # from indices to actual data subsets
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_validation = X[validation_idx]
    y_validation = y[validation_idx]

    # train Linear Regression on training data
    model.fit(X_train, y_train)

    # do prediction on the one validation data point
    yHat[validation_idx] = model.predict(X_validation)

  MSE = mean_squared_error(y, yHat)
  R2 = r2_score(y, yHat)

  return MSE, R2

Create a total of eight polynomials from degree 1 to 8. Compute and print training MSE and validation MSE values for each of them. Validation MSE should be computed using LOOCV.

In [ ]:
# Build polynomial models from degree 1 to 8 and evaluate them
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

degrees = list(range(1, 9))
train_mse = []
val_mse = []

for degree in degrees:
    # Train on the full dataset to get the training MSE
    full_model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree)),
        ("linreg", LinearRegression())
    ])
    full_model.fit(X, y)
    train_predictions = full_model.predict(X)
    train_error = mean_squared_error(y, train_predictions)
    train_mse.append(train_error)

    # Use LOOCV to estimate validation MSE for the same degree
    cv_model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree)),
        ("linreg", LinearRegression())
    ])
    validation_error, _ = loocv(X, y, cv_model)
    val_mse.append(validation_error)

    print(f"Degree {degree}: training MSE = {train_error:.4f}, validation MSE (LOOCV) = {validation_error:.4f}")

# Plot both curves so we can compare them
plt.figure(figsize=(8, 5))
plt.plot(degrees, train_mse, marker="o", label="Training MSE")
plt.plot(degrees, val_mse, marker="o", label="Validation MSE (LOOCV)")
plt.xlabel("Polynomial degree")
plt.ylabel("Mean Squared Error")
plt.xticks(degrees)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()
